In [1]:
# https://medium.com/@prasadmahamulkar/fine-tuning-phi-2-a-step-by-step-guide-e672e7f1d009

In [2]:
!pip install torch 
!pip install peft
!pip install bitsandbytes
!pip install transformers
!pip install trl 
!pip install accelerate
!pip install einops

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [4]:
base_model = "microsoft/Phi-3-mini-4k-instruct"
new_model = "phi-3-medquad"

# Dataset
dataset = load_dataset("prsdm/medquad-phi2-1k", split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# LoRA configuration
peft_config = LoraConfig(
    r= 64,          
    lora_alpha= 16,
    lora_dropout=0.05, #0.1
    bias="none",
    task_type="CAUSAL_LM",
    #target_modules= ["all-linear"] 
    target_modules="all-linear",
)

In [7]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
    
)

In [8]:
# SFTTraining Ref: https://www.superannotate.com/blog/llm-fine-tuning
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length= None,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Train model
trainer.train()

c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\trl\trainer\sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/250 [00:00<?, ?it/s]

c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


{'loss': 1.1829, 'grad_norm': 0.0995219349861145, 'learning_rate': 0.0001975746552556772, 'epoch': 0.1}
{'loss': 0.9496, 'grad_norm': 0.1282738298177719, 'learning_rate': 0.00018550053929480202, 'epoch': 0.2}
{'loss': 0.8866, 'grad_norm': 0.11445974558591843, 'learning_rate': 0.00016449948488669639, 'epoch': 0.3}
{'loss': 0.8332, 'grad_norm': 0.16387300193309784, 'learning_rate': 0.000136764169663272, 'epoch': 0.4}
{'loss': 0.8885, 'grad_norm': 0.11799024045467377, 'learning_rate': 0.00010519038181318999, 'epoch': 0.5}
{'loss': 0.8007, 'grad_norm': 0.16740044951438904, 'learning_rate': 7.307467669163655e-05, 'epoch': 0.6}
{'loss': 0.8293, 'grad_norm': 0.15509049594402313, 'learning_rate': 4.377019014049223e-05, 'epoch': 0.7}
{'loss': 0.8091, 'grad_norm': 0.17858301103115082, 'learning_rate': 2.03365443542764e-05, 'epoch': 0.8}
{'loss': 0.8547, 'grad_norm': 0.12917818129062653, 'learning_rate': 5.22039891260262e-06, 'epoch': 0.9}
{'loss': 0.7637, 'grad_norm': 0.15002545714378357, 'learn

TrainOutput(global_step=250, training_loss=0.879844841003418, metrics={'train_runtime': 2095.1778, 'train_samples_per_second': 0.477, 'train_steps_per_second': 0.119, 'total_flos': 1.996136183660544e+16, 'train_loss': 0.879844841003418, 'epoch': 1.0})

In [9]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [10]:
prompt = "What are the treatments for Gastrointestinal Carcinoid Tumors?"
instruction = f"### Instruction: {prompt} "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


 {eq}\textbf{Key Points}{/eq}:    - Surgery is the main treatment for gastrointestinal carcinoid tumors.    - The type of surgery depends on the location of the tumor.    - If the tumor has spread to other parts of the body, chemotherapy or radiation therapy may be used.    - Targeted therapy is being studied in clinical trials.    - New treatments are being tested in clinical trials.    - Patients may want to think about taking part in a clinical trial.    - Patients should talk with their doctor about which treatment is right for them.    - Patients should talk with their doctor about joining a clinical trial.    - Patients should think about taking part in a clinical trial.    - Patients should think
